In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# algorithms
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import dummy
from sklearn import linear_model
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
#import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
#import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# evaluation
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure  







/Users/blazingsun88/miniconda3/envs/ml_catboost/lib/python3.12/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:18: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [20]:
# Load the CSV file for the training dataset
data_train = pd.read_csv('train.csv')
df_train = pd.DataFrame(data_train)

df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [22]:
# Check if there are any missing values in the DataFrame
has_missing = data_train.isnull().values.any()
print("Are there any missing values?", has_missing)

#df_train.isna().sum()

# missing_summary = pd.DataFrame({
#     'missing_count': df_train.isnull().sum(),
#     'missing_percentage': df_train.isnull().mean() * 100
# })
# print(missing_summary)

# Only show columns with missing values
df_train.isnull().sum()[df_train.isnull().sum() > 0]


Are there any missing values? True


LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [23]:
df_train = df_train.drop('Id', axis=1)
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


### Removing non meaningful Nas

From these missing columns, NaN has meaning for some of them. The ones with no meaning are the ones I will remove. These are:

LotFrontage

Electrical

In [24]:
df_train = df_train.dropna(subset=['LotFrontage', 'Electrical'])

### Remove Outliers for random variables

In [25]:
random_var_outlier_num_cols = ['YearBuilt',
                   '1stFlrSF', 'FullBath',
                   'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
                    'MoSold', 'YrSold' ]

In [26]:
def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    
    for col in columns:
        # Calculate Q1, Q3, and IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Create mask for non-outliers
        mask = (df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)
        df_clean = df_clean[mask]
        
        print(f"{col}: removed {len(df) - len(df_clean)} outliers")
    
    return df_clean

# Remove outliers
df_train_no_outliers = remove_outliers_iqr(df_train, random_var_outlier_num_cols)

# Print shape comparison
print("\nShape comparison:")
print(f"Original: {df_train.shape}")
print(f"After outlier removal: {df_train_no_outliers.shape}")

YearBuilt: removed 0 outliers
1stFlrSF: removed 13 outliers
FullBath: removed 13 outliers
BedroomAbvGr: removed 40 outliers
KitchenAbvGr: removed 91 outliers
TotRmsAbvGrd: removed 103 outliers
MoSold: removed 103 outliers
YrSold: removed 103 outliers

Shape comparison:
Original: (1200, 80)
After outlier removal: (1097, 80)


In [27]:
#this is very confusing but im too lazy to change all the df_train names :'))
df_train = df_train_no_outliers

df_train.shape

(1097, 80)

### Remove outliers for columns where 0 can mean non existant

In [22]:
# Get numeric columns
numeric_cols = df_train.select_dtypes(include=['int64', 'float64']).columns


# Remove categorical and random_var_outlier_num_cols from numeric columns
remaining_numeric_cols = [col for col in numeric_cols 
                        if col not in random_var_outlier_num_cols 
                        and col != 'LotFrontage' and col != 'LotArea' and col != 'GrLivArea' and col != 'SalePrice' ]

print("Remaining numeric columns:")
print(remaining_numeric_cols)
print(f"\nNumber of remaining columns: {len(remaining_numeric_cols)}")

Remaining numeric columns:
['MSSubClass', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

Number of remaining columns: 25


### Columns with no outliers

In [10]:
fixed_num_cols = [
    'MSSubClass', 'OverallQual', 'OverallCond'
]

In [11]:
# Count values in fixed_num_cols to make sure there are no weird values
for col in fixed_num_cols:    
    print(f"Value counts for {col}:")
    print(df_train[col].value_counts())
    print("\n")




Value counts for MSSubClass:
MSSubClass
20     437
60     230
50     128
120     67
30      63
70      55
160     55
90      47
80      37
190     27
75      15
85      14
45      12
180     10
40       3
Name: count, dtype: int64


Value counts for OverallQual:
OverallQual
5     332
6     282
7     266
8     142
4      98
9      42
10     17
3      17
2       3
1       1
Name: count, dtype: int64


Value counts for OverallCond:
OverallCond
5    683
6    206
7    165
8     60
4     46
3     20
9     15
2      5
Name: count, dtype: int64




### Columns that could have outliers

In [12]:
remaining_numeric_cols = [col for col in remaining_numeric_cols if col not in fixed_num_cols]



In [13]:
def calculate_modified_zscore(series):
    filtered = series[series != 0]
    if len(filtered) == 0:
        return pd.Series(np.zeros(len(series)), index=series.index)
    
    median = np.median(filtered)
    mad = np.median(np.abs(filtered - median))
    modified_zscore = 0.6745 * (series - median) / mad if mad != 0 else 0
    return modified_zscore

# Create copy for cleaning
df_clean = df_train.copy()

for col in remaining_numeric_cols:
    # Calculate Modified Z-score on filtered data
    filtered_data = df_clean[df_clean[col] != 0][col]
    df_clean[f'{col}_mod_zscore'] = calculate_modified_zscore(filtered_data)
    
    # Identify outliers
    outlier_mask = abs(df_clean[f'{col}_mod_zscore']) > 3.5
    outliers = df_clean[outlier_mask]
    
    # Remove outliers
    df_clean = df_clean[~outlier_mask]
    
    print(f"\n{col}:")
    print(f"Number of outliers removed: {len(outliers)}")
    print(f"Remaining rows: {len(df_clean)}")
    
    # # Plot before/after distribution
    # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # # Before removal
    # ax1.scatter(range(len(filtered_data)), filtered_data)
    # ax1.set_title(f'{col} Before Outlier Removal')
    
    # # After removal
    # cleaned_data = df_clean[df_clean[col] != 0][col]
    # ax2.scatter(range(len(cleaned_data)), cleaned_data)
    # ax2.set_title(f'{col} After Outlier Removal')
    
    # plt.tight_layout()
    # plt.show()


YearRemodAdd:
Number of outliers removed: 0
Remaining rows: 1200

MasVnrArea:
Number of outliers removed: 0
Remaining rows: 1200

BsmtFinSF1:
Number of outliers removed: 6
Remaining rows: 1194

BsmtFinSF2:
Number of outliers removed: 1
Remaining rows: 1193

BsmtUnfSF:
Number of outliers removed: 5
Remaining rows: 1188

TotalBsmtSF:
Number of outliers removed: 5
Remaining rows: 1183

2ndFlrSF:
Number of outliers removed: 3
Remaining rows: 1180

LowQualFinSF:
Number of outliers removed: 0
Remaining rows: 1180

BsmtFullBath:
Number of outliers removed: 0
Remaining rows: 1180

BsmtHalfBath:
Number of outliers removed: 0
Remaining rows: 1180

HalfBath:
Number of outliers removed: 0
Remaining rows: 1180

Fireplaces:
Number of outliers removed: 0
Remaining rows: 1180

GarageYrBlt:
Number of outliers removed: 0
Remaining rows: 1180

GarageCars:
Number of outliers removed: 0
Remaining rows: 1180

GarageArea:
Number of outliers removed: 3
Remaining rows: 1177

WoodDeckSF:
Number of outliers rem

In [14]:
df_clean

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,GarageYrBlt_mod_zscore,GarageCars_mod_zscore,GarageArea_mod_zscore,WoodDeckSF_mod_zscore,OpenPorchSF_mod_zscore,EnclosedPorch_mod_zscore,3SsnPorch_mod_zscore,ScreenPorch_mod_zscore,PoolArea_mod_zscore,MiscVal_mod_zscore
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,0.372138,NaN,0.024089,NaN,NaN,NaN,NaN,NaN
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,0,-0.139552,1.638972,NaN,NaN,NaN,NaN,NaN,NaN
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,0,0.721017,NaN,-0.433607,NaN,NaN,NaN,NaN,NaN
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,0,0.918716,NaN,-0.602232,1.798667,NaN,NaN,NaN,NaN
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,0,2.046759,0.302579,0.578143,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,20,RL,90.0,17217,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,NaN,-1.664187,-0.096357,NaN,NaN,NaN,NaN,NaN
1454,20,FV,62.0,7500,Pave,Pave,Reg,Lvl,AllPub,Inside,...,NaN,0,-0.488431,NaN,1.276732,NaN,NaN,NaN,NaN,NaN
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,-0.139552,NaN,-0.481786,NaN,NaN,NaN,NaN,NaN
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,0,0.093034,2.281953,NaN,NaN,NaN,NaN,NaN,NaN


### Making the actual clean dataframe

In [15]:
# Find common indices
common_indices = df_train_no_outliers.index.intersection(df_clean.index)

# Create new dataframe with common rows
df_final = df_train_no_outliers.loc[common_indices]

# Print shape information
print("Original shapes:")
print(f"df_train_no_outliers: {df_train_no_outliers.shape}")
print(f"df_clean: {df_clean.shape}")
print(f"\nFinal shape after intersection: {df_final.shape}")

Original shapes:
df_train_no_outliers: (1097, 80)
df_clean: (1117, 102)

Final shape after intersection: (1036, 80)


In [16]:
df_train = df_final

## Changing all columns to numerical quantities

In [28]:
#Change columns with 'Yes'/'No' to 1/0
yes_no_cols = [col for col in df_train.columns if set(df_train[col].unique()) == {'Yes', 'No'}]
for col in yes_no_cols:
    df_train[col] = df_train[col].map({'Yes': 1, 'No': 0})

# One-hot encode categorical variables


# Identify categorical and object columns
cat_cols = df_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Categorical columns to encode: {cat_cols}")
print(f"Number of categorical columns: {len(cat_cols)}")

# Initialize OneHotEncoder
# sparse_output=False returns a dense array instead of sparse matrix
# handle_unknown='ignore' will ignore unknown categories during transform
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit and transform the categorical columns
encoded_array = encoder.fit_transform(df_train[cat_cols])

# Get feature names for the encoded columns
encoded_feature_names = encoder.get_feature_names_out(cat_cols)

# Create a dataframe with the encoded features
df_encoded = pd.DataFrame(
    encoded_array, 
    columns=encoded_feature_names,
    index=df_train.index
)

# Drop original categorical columns and concatenate encoded columns
df_train_encoded = df_train.drop(columns=cat_cols)
df_train_encoded = pd.concat([df_train_encoded, df_encoded], axis=1)

# Convert boolean columns to integers (if any remain)
bool_cols = df_train_encoded.select_dtypes(include='bool').columns
if len(bool_cols) > 0:
    df_train_encoded[bool_cols] = df_train_encoded[bool_cols].astype(int)

print(f"\nOriginal shape: {df_train.shape}")
print(f"Encoded shape: {df_train_encoded.shape}")
print(f"\nFirst few rows:")
df_train_encoded.head()




Categorical columns to encode: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
Number of categorical columns: 43

Original shape: (1097, 80)
Encoded shape: (1097, 290)

First few rows:


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1097 entries, 0 to 1459
Columns: 290 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(256), int64(34)
memory usage: 2.4 MB


## Doing Min Max Scaling

In [18]:
# Initialize scaler
scaler = MinMaxScaler()

# Create copy and drop SalePrice temporarily
X = df_train_encoded.drop('SalePrice', axis=1)
y = df_train_encoded['SalePrice']

# Scale features
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns
)

# Scale SalePrice separately
y_scaled = pd.DataFrame(
    scaler.fit_transform(y.values.reshape(-1, 1)),
    columns=['SalePrice']
)

In [21]:
# setting up the data and target variables
data = X_scaled
target = y_scaled

# Splitting the training data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

In [23]:
y_train = y_train.values.ravel()

### Correlation Analysis

In [ ]:


# Initialize scaler
scaler = MinMaxScaler()

# Create copy and drop SalePrice temporarily
X = df_train_encoded.drop('SalePrice', axis=1)
y = df_train_encoded['SalePrice']

# Scale features
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns
)

# Scale SalePrice separately
y_scaled = pd.DataFrame(
    scaler.fit_transform(y.values.reshape(-1, 1)),
    columns=['SalePrice']
)
# First identify columns with zero variance
zero_var_cols = []
valid_cols = []

for col in X_scaled.columns:
    if X_scaled[col].std() == 0:
        zero_var_cols.append(col)
    else:
        valid_cols.append(col)

print("Columns with zero variance (all identical values):")
for col in zero_var_cols:
    print(f"{col}: unique value = {X_scaled[col].iloc[0]}")



# Now calculate regression only for valid columns
results = []
for col in valid_cols:
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        X_scaled[col], 
        y_scaled['SalePrice']
    )
    r_squared = r_value**2
    results.append({
        'Column': col,
        'R-squared': r_squared,
        'p-value': p_value
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('R-squared', ascending=False)
# print("\nRegression results for valid columns:")
# print(results_df.to_string(float_format=lambda x: '{:.6f}'.format(x)))

# Filter for significant p-values
significant_linear_results = results_df[results_df['p-value'] <= 0.05].copy()

# Sort by R-squared value
significant_linear_results = significant_linear_results.sort_values('R-squared', ascending=False)

# Display results
print("Columns with significant p-values (p ≤ 0.05):")
print(significant_linear_results.to_string(float_format=lambda x: '{:.6f}'.format(x)))

# Store significant column names
significant_linear_cols = significant_linear_results['Column'].tolist()
print(f"\nNumber of significant features: {len(significant_linear_cols)}")

In [ ]:
# Calculate Spearman's r for each column
results = []
for col in X_scaled.columns:
    # Calculate Spearman correlation
    correlation, p_value = stats.spearmanr(X_scaled[col], y_scaled['SalePrice'])
    results.append({
        'Column': col,
        'Spearman_r': correlation,
        'p-value': p_value
    })

# Convert to dataframe and sort by correlation strength
results_df = pd.DataFrame(results)
results_df['abs_correlation'] = abs(results_df['Spearman_r'])
results_df = results_df.sort_values('abs_correlation', ascending=False)
results_df = results_df.drop('abs_correlation', axis=1)

# # Display results
# pd.set_option('display.max_rows', None)
# print(results_df.to_string(float_format=lambda x: '{:.6f}'.format(x)))

# Filter for significant Spearman correlations
spearman_significant = results_df[results_df['p-value'] <= 0.05].copy()

# Sort by absolute correlation value
spearman_significant['abs_correlation'] = abs(spearman_significant['Spearman_r'])
spearman_significant = spearman_significant.sort_values('abs_correlation', ascending=False)
spearman_significant = spearman_significant.drop('abs_correlation', axis=1)

# # Display results
# print("Features with significant Spearman correlations (p ≤ 0.05):")
# print(spearman_significant.to_string(float_format=lambda x: '{:.6f}'.format(x)))

# Store significant column names
significant_spearman_cols = spearman_significant['Column'].tolist()
print(f"\nNumber of significant features: {len(significant_spearman_cols)}")

# Filter for strong correlations (|r| >= 0.1) from significant results
strong_spearman_corr = spearman_significant[abs(spearman_significant['Spearman_r']) >= 0.1].copy()

# Sort by absolute correlation value
strong_spearman_corr['abs_correlation'] = abs(strong_spearman_corr['Spearman_r'])
strong_spearman_corr = strong_spearman_corr.sort_values('abs_correlation', ascending=False)
strong_spearman_corr = strong_spearman_corr.drop('abs_correlation', axis=1)

# Display results
print("Features with strong Spearman correlations (|r| ≥ 0.1 and p ≤ 0.05):")
print(strong_spearman_corr.to_string(float_format=lambda x: '{:.6f}'.format(x)))

# Store filtered column names
strong_spearman_corr_cols = strong_spearman_corr['Column'].tolist()
print(f"\nNumber of strong correlation features: {len(strong_spearman_corr_cols)}")

### Choose only significant columns from both analyses

In [ ]:
df_train_pearson = df_train_encoded[significant_linear_cols + ['SalePrice']]
df_train_pearson.info()

In [ ]:
df_train_spearman = df_train_encoded[strong_spearman_corr_cols + ['SalePrice']]
df_train_spearman.info()

## ยังไม่ได้ทำำำำำ Applying the same method to the test data.

In [32]:
data_test = pd.read_csv('test.csv')
df_test = pd.DataFrame(data_test)

df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [33]:
df_test = df_test.drop('Id', axis=1)
df_test = df_test.dropna(subset=['LotFrontage', 'Electrical'])

# Apply Yes/No mapping to test data
for col in yes_no_cols:
    df_test[col] = df_test[col].map({'Yes': 1, 'No': 0})

# Transform test data using the fitted encoder
encoded_test_array = encoder.transform(df_test[cat_cols])

# Create DataFrame with encoded features
df_encoded = pd.DataFrame(
    encoded_test_array,
    columns=encoded_feature_names,
    index=df_test.index
)

# Drop original categorical columns and concatenate encoded columns
df_test_encoded = df_test.drop(columns=cat_cols)
df_test_encoded = pd.concat([df_test_encoded, df_encoded], axis=1)

# Convert boolean columns to integers (if any)
if len(bool_cols) > 0:
    df_test_encoded[bool_cols] = df_test_encoded[bool_cols].astype(int)

print(f"Original test shape: {df_test.shape}")
print(f"Encoded test shape: {df_test_encoded.shape}")


Original test shape: (1232, 79)
Encoded test shape: (1232, 289)


# Training Numeric Models :0

## XGBoost

### On significant_pearson

In [64]:
# setting up the data and target variables
data = df_train_pearson.drop('SalePrice', axis=1)
target = df_train_pearson['SalePrice']

# Splitting the training data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

### On significant_spearman

In [58]:
# setting up the data and target variables
data = df_train_spearman.drop('SalePrice', axis=1)
target = df_train_spearman['SalePrice']

# Splitting the training data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

In [ ]:
X_train.info()

In [ ]:
# initialize the model
model = xgb.XGBRegressor(objective='reg:squaredlogerror')  
# fit the model
model.fit(X_train, y_train)
# make predictions
y_pred = model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

### On all encoded data

In [34]:
# setting up the data and target variables
data = df_train_encoded.drop('SalePrice', axis=1)
target = df_train_encoded['SalePrice']

# Splitting the training data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987 entries, 1217 to 1136
Columns: 289 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(256), int64(33)
memory usage: 2.2 MB


In [34]:
# initialize the model
model = xgb.XGBRegressor(objective='reg:squarederror')  
# fit the model
model.fit(X_train, y_train)
# make predictions
y_pred = model.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.2f}")

Mean Squared Error: 452317216.00
Root Mean Squared Error: 21267.75
Mean Absolute Error: 14238.94
R² Score: 0.88


### Dummy Regressor

In [ ]:
dum = dummy.DummyRegressor()
dum.fit(X_train, y_train)
y_pred = dum.predict(X_test)
y_pred


In [ ]:
# Store the actual data and the predictions in a DataFrame for later
price_test = pd.DataFrame()
price_test["Actual_price"] = y_test
price_test["Dummy_price"] = y_pred

price_test

In [ ]:
# Baseline: Dummy Regressor
print("\n" + "="*50)
print("DUMMY REGRESSOR (Baseline)")
print("="*50)
dum = dummy.DummyRegressor()
dum.fit(X_train, y_train)
y_pred_dummy = dum.predict(X_test)

dummy_mse = metrics.mean_squared_error(y_test, y_pred_dummy)
dummy_rmse = np.sqrt(dummy_mse)
dummy_r2 = metrics.r2_score(y_test, y_pred_dummy)

print(f"Dummy MSE:  {dummy_mse:,.2f}")
print(f"Dummy RMSE: {dummy_rmse:,.2f}")
print(f"Dummy R²:   {dummy_r2:.4f}")

### Just comparing to see which one is better

In [21]:
# HistGradientBoostingRegressor (handles NaN natively)
print("\n" + "="*50)
print("HIST GRADIENT BOOSTING REGRESSOR")
print("="*50)

hgb = HistGradientBoostingRegressor(
    max_iter=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

hgb.fit(X_train, y_train)
y_pred_hgb = hgb.predict(X_test)

hgb_mse = metrics.mean_squared_error(y_test, y_pred_hgb)
hgb_rmse = np.sqrt(hgb_mse)
hgb_r2 = metrics.r2_score(y_test, y_pred_hgb)
hgb_mae = metrics.mean_absolute_error(y_test, y_pred_hgb)

print(f"HGB MSE:  {hgb_mse:,.2f}")
print(f"HGB RMSE: {hgb_rmse:,.2f}")
print(f"HGB MAE:  {hgb_mae:,.2f}")
print(f"HGB R²:   {hgb_r2:.4f}")

# print("\n" + "="*50)
# print("LIGHTGBM REGRESSOR")
# print("="*50)

# # Initialize LightGBM
# lgbm = lgb.LGBMRegressor(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     random_state=42,
#     verbose=-1
# )

# # Fit the model
# lgbm.fit(X_train, y_train)
# y_pred_lgbm = lgbm.predict(X_test)

# # Calculate metrics
# lgbm_mse = metrics.mean_squared_error(y_test, y_pred_lgbm)
# lgbm_rmse = np.sqrt(lgbm_mse)
# lgbm_r2 = metrics.r2_score(y_test, y_pred_lgbm)
# lgbm_mae = metrics.mean_absolute_error(y_test, y_pred_lgbm)

# # Print results
# print(f"LGBM MSE:  {lgbm_mse:,.2f}")
# print(f"LGBM RMSE: {lgbm_rmse:,.2f}")
# print(f"LGBM MAE:  {lgbm_mae:,.2f}")
# print(f"LGBM R²:   {lgbm_r2:.4f}")



HIST GRADIENT BOOSTING REGRESSOR
HGB MSE:  390,419,998.29
HGB RMSE: 19,759.05
HGB MAE:  13,903.37
HGB R²:   0.8986


In [35]:
from catboost import CatBoostRegressor

print("\n" + "="*50)
print("CATBOOST REGRESSOR")
print("="*50)

# Initialize CatBoost
cat = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=5,
    random_seed=42,
    verbose=False
)

# Fit the model
cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)

# Calculate metrics
cat_mse = metrics.mean_squared_error(y_test, y_pred_cat)
cat_rmse = np.sqrt(cat_mse)
cat_r2 = metrics.r2_score(y_test, y_pred_cat)
cat_mae = metrics.mean_absolute_error(y_test, y_pred_cat)

# Print results
print(f"CAT MSE:  {cat_mse:,.2f}")
print(f"CAT RMSE: {cat_rmse:,.2f}")
print(f"CAT MAE:  {cat_mae:,.2f}")
print(f"CAT R²:   {cat_r2:.4f}")


CATBOOST REGRESSOR
CAT MSE:  494,414,324.35
CAT RMSE: 22,235.43
CAT MAE:  14,866.47
CAT R²:   0.8996


## Hyperparameter tuning

### Catboost

In [37]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import numpy as np
from scipy.stats import uniform, randint

# Define parameter grid
param_dist = {
    'iterations': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.3),
    'depth': randint(3, 10),
    'l2_leaf_reg': uniform(1, 10),
    'bagging_temperature': uniform(0, 1),
    'random_strength': uniform(1, 20)
}

# Initialize base model
cat = CatBoostRegressor(
    verbose=False,
    random_seed=42
)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=cat,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    random_state=42,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get best parameters and score
print("\n" + "="*50)
print("CATBOOST BEST PARAMETERS")
print("="*50)
print(f"Best parameters: {random_search.best_params_}")
print(f"Best CV score: {-random_search.best_score_:.4f} MSE")

# Train final model with best parameters
best_cat = CatBoostRegressor(**random_search.best_params_, random_seed=42, verbose=False)
best_cat.fit(X_train, y_train)
y_pred = best_cat.predict(X_test)

# Calculate metrics
cat_mse = metrics.mean_squared_error(y_test, y_pred)
cat_rmse = np.sqrt(cat_mse)
cat_r2 = metrics.r2_score(y_test, y_pred)
cat_mae = metrics.mean_absolute_error(y_test, y_pred)

# Print results
print("\n" + "="*50)
print("OPTIMIZED CATBOOST PERFORMANCE")
print("="*50)
print(f"CAT MSE:  {cat_mse:,.2f}")
print(f"CAT RMSE: {cat_rmse:,.2f}")
print(f"CAT MAE:  {cat_mae:,.2f}")
print(f"CAT R²:   {cat_r2:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits

CATBOOST BEST PARAMETERS
Best parameters: {'bagging_temperature': np.float64(0.020584494295802447), 'depth': 4, 'iterations': 443, 'l2_leaf_reg': np.float64(9.324426408004218), 'learning_rate': np.float64(0.07370173320348283), 'random_strength': np.float64(4.636499344142012)}
Best CV score: 490632174.6899 MSE

OPTIMIZED CATBOOST PERFORMANCE
CAT MSE:  427,537,122.01
CAT RMSE: 20,676.97
CAT MAE:  13,656.00
CAT R²:   0.9132
